In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window
import time

### Data Reading

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databricks1ete.dfs.core.windows.net/customers")

In [0]:
display(df)

In [0]:
df = df.drop('_rescued_data')
display(df)

In [0]:
df = df.withColumn("domain", split(col("email"), "@")[1])
display(df)

In [0]:
df.groupBy("domain").agg(count("customer_id").alias("total_customers")).sort("total_customers",ascending=False).display()


In [0]:
df_gmail = df.filter(col('domain') == "gmail.com")
display(df_gmail)
time.sleep(10)

df_yahoo = df.filter(col('domain') == "yahoo.com")
display(df_yahoo)
time.sleep(10)

df_hotmail = df.filter(col('domain') == "hotmail.com")
display(df_hotmail)
time.sleep(10)

In [0]:
df = df.withColumn("full_name" , concat(col("first_name"),lit(" "),col("last_name")))
df = df.drop("first_name","last_name")
display(df)

### Data Writing

In [0]:
df.write.format("delta").mode("append").save("abfss://silver@databricks1ete.dfs.core.windows.net/customers")